In [ ]:
# Cell 1: imports & path setup
import sys
from pathlib import Path
import numpy as np
import GPy
import matplotlib.pyplot as plt
repo_root = Path.cwd().parents[0] if (Path.cwd() / 'notebooks').exists() else Path.cwd()
functions_path = repo_root / 'functions'
sys.path.append(str(functions_path.resolve()))
from monotonic_gp_utils import gp_monotonic
print("Imported gp_monotonic from:", functions_path.resolve())

In [ ]:
# Cell 2: generate synthetic data
np.random.seed(42)
X = np.sort(np.random.uniform(0.1, 6, 50))[:, None]
Y = np.where(X < 3, 0.0, 0.5) + 0.05 * np.random.randn(*X.shape)

plt.figure(figsize=(6, 3))
plt.plot(X, Y, 'kx')
plt.title('Training data (step-like)')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


In [ ]:
# Cell 3: base GP and monotonic GP
kernel = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=2.0)
base_model = GPy.models.GPRegression(X, Y, kernel)

mono_model = gp_monotonic(base_model, X, Y,
                          nvd=[1.0],
                          nu=1e-8,
                          nv=10,
                          force=True,
                          display=True,
                          optimize='on',
                          init='sample')


In [ ]:
# Cell 4: standard GP for comparison
standard_gp = GPy.models.GPRegression(X, Y, kernel.copy())
standard_gp.Gaussian_noise.variance = 0.01
standard_gp.optimize(messages=True)


In [ ]:
# Cell 5: predictions, plotting and saving result
Xtest = np.linspace(0.1, 6, 200)[:, None]
mu_mono, var_mono = mono_model.predict(Xtest, full_cov=False, include_likelihood=False)
mu_standard, var_standard = standard_gp.predict(Xtest, full_cov=False, include_likelihood=False)

# Flatten / ensure shapes
mu_mono = mu_mono[:, 0]
var_mono = np.clip(var_mono.squeeze(), 1e-6, None)
mu_standard = mu_standard[:, 0]
var_standard = np.clip(var_standard.squeeze(), 1e-6, None)

plt.figure(figsize=(10, 6))
plt.plot(X, Y, 'kx', label="Training Data (Step Function)")

plt.plot(Xtest, mu_mono, '-', label="Monotonic GP Mean")
plt.fill_between(Xtest.ravel(),from pathlib import Path
import os

# --- Save plot safely in repo structure ---
repo_root = Path.cwd().parents[0] if (Path.cwd() / 'notebooks').exists() else Path.cwd()
results_dir = repo_root / 'results'

# Make sure 'results' folder exists
os.makedirs(results_dir, exist_ok=True)

# Save the figure
plot_path = results_dir / 'result_plot.png'
plt.savefig(plot_path, dpi=200)
plt.show()

print(f"✅ Saved plot to: {plot_path.resolve()}")

                 mu_mono - 2 * np.sqrt(var_mono),
                 mu_mono + 2 * np.sqrt(var_mono),
                 alpha=0.2, label="Monotonic GP 95% CI")

plt.plot(Xtest, mu_standard, '--', label="Standard GP Mean")
plt.fill_between(Xtest.ravel(),
                 mu_standard - 2 * np.sqrt(var_standard),
                 mu_standard + 2 * np.sqrt(var_standard),
                 alpha=0.2, label="Standard GP 95% CI")

plt.title("Monotonic GP vs. Standard GP (Step Function)")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.tight_layout()

from pathlib import Path
import os
repo_root = Path.cwd().parents[0] if (Path.cwd() / 'notebooks').exists() else Path.cwd()
results_dir = repo_root / 'results'
os.makedirs(results_dir, exist_ok=True)
plot_path = results_dir / 'result_plot.png'
plt.savefig(plot_path, dpi=200)
plt.show()
print(f"Saved plot to: {plot_path.resolve()}")
